# project #4

# Step 3: Dropout Regularization in Keras

In [2]:
# Baseline Model on the Sonar Dataset
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataframe = read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# baseline
def create_baseline():
                        # create model
                        model = Sequential()
                        model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
                        model.add(Dense(30, kernel_initializer='normal', activation='relu'))
                        model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
                        # Compile model
                        sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
                        model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                        return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/hp/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Baseline: 82.14% (7.30%)


# Step 4: Using Dropout on the Visible Layer

In [3]:
# dropout in the input layer with weight constraint
def create_model():
                                # create model
                                model = Sequential()
                                model.add(Dropout(0.2, input_shape=(60,)))
                                model.add(Dense(60, activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dense(30,  activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
                                # Compile model
                                sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                                model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                                return model
                            
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 86.49% (8.00%)


# Step 5: Trying to Improve Performance

In [20]:
# dropout in the input layer with weight constraint
def create_model():
                                # create model
                                model = Sequential()
                                model.add(Dropout(0.2, input_shape=(60,)))
                                model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dense(15,  activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dense(1, activation='sigmoid'))
                                # Compile model
                                sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                                model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                                return model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 87.51% (3.95%)


# Step 6: Using Dropout on Hidden Layers

In [22]:
# dropout in hidden layers with weight constraint
def create_model():
                            # create model, insert code here
                            model = Sequential()
                            model.add(Dense(60, input_dim=60,activation='relu', kernel_constraint=maxnorm(3)))
                            model.add(Dropout(0.2))
                            model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
                            model.add(Dropout(0.2))
                            model.add(Dense(1,activation='sigmoid'))
                            # Compile model
                            sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                            model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                            return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 82.17% (3.40%)


# Step 7: Trying to Improve Performance

In [14]:
# dropout in hidden layers with weight constraint
def create_model():
                            # create model, insert code here
                            model = Sequential()
                            model.add(Dense(30, input_dim=60,activation='relu', kernel_constraint=maxnorm(3)))
                            model.add(Dropout(0.15))
                            model.add(Dense(15, activation='relu', kernel_constraint=maxnorm(3)))
                            model.add(Dropout(0.15))
                            model.add(Dense(1,activation='sigmoid'))
                            # Compile model
                            sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                            model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                            return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 84.55% (5.48%)


# Step 8.1: Try Different Dropout values

In [17]:
# dropout in the input layer with weight constraint
def Different_Dropout_values():
                                # create model
                                model = Sequential()
                                model.add(Dense(60, input_dim=60,activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dropout(0.1))
                                model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dropout(0.1))
                                model.add(Dense(1,activation='sigmoid'))
                                # Compile model
                                sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                                model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                                return model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=Different_Dropout_values, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 83.62% (3.01%)


# Step 8.2: Try using a Larger network

In [18]:
# dropout in the input layer with weight constraint
def larger():
                                        # create model
                                        model = Sequential()
                                        model.add(Dense(80, input_dim=60,activation='relu', kernel_constraint=maxnorm(3)))
                                        model.add(Dropout(0.2))
                                        model.add(Dense(60, activation='relu', kernel_constraint=maxnorm(3)))
                                        model.add(Dropout(0.2))
                                        model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
                                        model.add(Dropout(0.2))
                                        model.add(Dense(1,activation='sigmoid'))
                                        # Compile model
                                        sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                                        model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                                        return model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=Different_Dropout_values, epochs=200, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 85.06% (3.30%)


# Step 8.3: Try using Dropout on both visible and hidden units

In [21]:
def dropOut_on_visible_and_hidden():
                    model = Sequential()
                    model.add(Dropout(0.2, input_shape=(60,)))
                    model.add(Dense(60, activation='relu', kernel_constraint=maxnorm(3)))
                    model.add(Dropout(0.20))
                    model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
                    model.add(Dropout(0.20))
                    model.add(Dense(1,activation='sigmoid'))
                    # Compile model
                    sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
                    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                    return model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=Different_Dropout_values, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 83.17% (2.11%)


# Step 8.4: Try using large learning rate with decay and larger momentum

In [28]:
# dropout in the input layer with weight constraint
def Different_Dropout_values():
                                # create model
                                model = Sequential()
                                model.add(Dense(60, input_dim=60,activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dropout(0.1))
                                model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
                                model.add(Dropout(0.1))
                                model.add(Dense(1,activation='sigmoid'))
                                # Compile model
                                sgd = SGD(lr=1, momentum=0.99, decay=1, nesterov=False)
                                model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                                return model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=Different_Dropout_values, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 49.48% (3.89%)


# Step 8.5: Try constraining the size of the network weights

In [62]:
# dropout in hidden layers with weight constraint
def create_model():
                            # create model, insert code here
                            model = Sequential()
                            model.add(Dense(30, input_dim=60,activation='relu', kernel_constraint=maxnorm(5)))
                            model.add(Dropout(0.2))
                            model.add(Dense(15, activation='relu', kernel_constraint=maxnorm(5)))
                            model.add(Dropout(0.2))
                            model.add(Dense(1,activation='sigmoid'))
                            # Compile model
                            sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)
                            model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
                            return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=80, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 88.04%(4.63%)


# Step 9: Read More Resources on Dropout, and try implementing them

In [67]:
# dropout in hidden layers with weight constraint
def create_model():
                            # create model, insert code here
                            model = Sequential()
                            model.add(Dropout(0.8, input_shape=(60,)))
                            model.add(Dense(60,activation='relu', kernel_constraint=maxnorm(5)))
                            model.add(Dropout(0.5))
                            model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(5)))
                            model.add(Dropout(0.5))
                            model.add(Dense(1,activation='sigmoid'))
                            # Compile model
                            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
                            return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=80, batch_size=8, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 76.98% (5.62%)
